<div class="alert alert-block alert-info">
<h1 style="text-align:center;font-family:avenir;">ICA first</h1>
Lou Planchamp & Victor Férat
</div>

In [1]:
%matplotlib inline

In [36]:
from nilearn import plotting
from nilearn.plotting import plot_stat_map
import pandas as pd
from nilearn.glm.first_level import FirstLevelModel
from numpy import array
import os
from nilearn.plotting import plot_design_matrix
import numpy as np
import matplotlib.pyplot as plt
from nilearn.glm.second_level import SecondLevelModel

from os.path import join
from bids import BIDSLayout
from bids.tests import get_test_data_path
from nilearn.plotting import plot_prob_atlas
from nilearn.image import iter_img
from nilearn.plotting import plot_stat_map, show


from nilearn.decomposition import CanICA

Here, thanks to BIDSLayout, we define whitch files we want to work on (all the resting states) and whitch files to use for the preprocessing (confounds)

In [41]:
bids_folder = '/home/carole/Bureau/BIDS_fMRIprep'
layout = BIDSLayout(bids_folder, derivatives=True)
layout

BIDS Layout: ...me/carole/Bureau/BIDS_fMRIprep | Subjects: 5 | Sessions: 10 | Runs: 11

nuisance model fund on https://carpentries-incubator.github.io/SDC-BIDS-fMRI/05-data-cleaning-with-nilearn/index.html

In [12]:
import os
from nilearn import image as nimg
from nilearn import plotting as nplot
#import matplotilb.pyplot as plt
import numpy as np
import nibabel as nib
%matplotlib inline

In [10]:
import pandas as pd

In [42]:
sub = '01'

In [44]:
func_files = layout.get(subject=sub, session='02', run='01',
                        datatype='func', task='rest',
                        desc='preproc',
                        space='MNIPediatricAsym',
                        extension='nii.gz',
                       return_type='file')

mask_files = layout.get(subject=sub, session='02', run='01',
                        datatype='func', task='rest',
                        desc='brain',
                        suffix='mask',
                        space='MNIPediatricAsym',
                        extension="nii.gz",
                       return_type='file')
 
confound_files = layout.get(subject=sub, session='02', run='01',
                            datatype='func', task='rest',
                            desc='confounds',
                           extension="tsv",
                           return_type='file')

In [46]:
func_file = func_files[0]
mask_file = mask_files[0]
confound_file = confound_files[0]

In [48]:
#Delimiter is \t --> tsv is a tab-separated spreadsheet
confound_df = pd.read_csv(confound_file, delimiter='\t')
confound_df.head()

,global_signal,global_signal_derivative1,global_signal_derivative1_power2,global_signal_power2,csf,csf_derivative1,csf_derivative1_power2,csf_power2,white_matter,white_matter_derivative1,...,motion_outlier69,motion_outlier70,motion_outlier71,motion_outlier72,motion_outlier73,motion_outlier74,motion_outlier75,motion_outlier76,motion_outlier77,motion_outlier78
0,639.415931,NaN,NaN,408852.733337,760.600087,NaN,NaN,578512.492765,661.915190,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,635.508164,-3.907767,15.270645,403870.626640,752.670444,-7.929643,62.879239,566512.797591,659.286386,-2.628805,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,635.143267,-0.364897,0.133150,403406.969496,755.056716,2.386272,5.694294,570110.644804,659.318649,0.032264,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,632.560471,-2.582796,6.670835,400132.749503,743.011377,-12.045339,145.090187,552065.907076,656.313738,-3.004912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,634.725608,2.165137,4.687817,402876.597045,746.494014,3.482636,12.128754,557253.312375,658.056136,1.742398,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
confound_vars =['csf', 
              'white_matter', 
              'trans_x', 'trans_x_derivative1', 'trans_x_derivative1_power2', 
              'trans_y', 'trans_y_derivative1', 'trans_y_derivative1_power2', 
              'trans_z', 'trans_z_derivative1', 'trans_z_derivative1_power2', 
              'rot_x', 'rot_x_derivative1', 'rot_x_derivative1_power2', 
              'rot_y', 'rot_y_derivative1', 'rot_y_derivative1_power2', 
              'rot_z', 'rot_z_derivative1', 'rot_z_derivative1_power2'
             ]

In [50]:
# Get derivative column names
derivative_columns = ['{}_derivative1'.format(c) for c
                     in confound_vars]

print(derivative_columns)

['csf_derivative1', 'white_matter_derivative1', 'trans_x_derivative1', 'trans_x_derivative1_derivative1', 'trans_x_derivative1_power2_derivative1', 'trans_y_derivative1', 'trans_y_derivative1_derivative1', 'trans_y_derivative1_power2_derivative1', 'trans_z_derivative1', 'trans_z_derivative1_derivative1', 'trans_z_derivative1_power2_derivative1', 'rot_x_derivative1', 'rot_x_derivative1_derivative1', 'rot_x_derivative1_power2_derivative1', 'rot_y_derivative1', 'rot_y_derivative1_derivative1', 'rot_y_derivative1_power2_derivative1', 'rot_z_derivative1', 'rot_z_derivative1_derivative1', 'rot_z_derivative1_power2_derivative1']


In [57]:
final_confounds = confound_vars + derivative_columns
print(final_confounds)

['csf', 'white_matter', 'trans_x', 'trans_x_derivative1', 'trans_x_derivative1_power2', 'trans_y', 'trans_y_derivative1', 'trans_y_derivative1_power2', 'trans_z', 'trans_z_derivative1', 'trans_z_derivative1_power2', 'rot_x', 'rot_x_derivative1', 'rot_x_derivative1_power2', 'rot_y', 'rot_y_derivative1', 'rot_y_derivative1_power2', 'rot_z', 'rot_z_derivative1', 'rot_z_derivative1_power2', 'csf_derivative1', 'white_matter_derivative1', 'trans_x_derivative1', 'trans_x_derivative1_derivative1', 'trans_x_derivative1_power2_derivative1', 'trans_y_derivative1', 'trans_y_derivative1_derivative1', 'trans_y_derivative1_power2_derivative1', 'trans_z_derivative1', 'trans_z_derivative1_derivative1', 'trans_z_derivative1_power2_derivative1', 'rot_x_derivative1', 'rot_x_derivative1_derivative1', 'rot_x_derivative1_power2_derivative1', 'rot_y_derivative1', 'rot_y_derivative1_derivative1', 'rot_y_derivative1_power2_derivative1', 'rot_z_derivative1', 'rot_z_derivative1_derivative1', 'rot_z_derivative1_po

In [69]:
confound_df = pd.DataFrame(columns=final_confounds)
confound_df.head()

,csf,white_matter,trans_x,trans_x_derivative1,trans_x_derivative1_power2,trans_y,trans_y_derivative1,trans_y_derivative1_power2,trans_z,trans_z_derivative1,...,trans_z_derivative1_power2_derivative1,rot_x_derivative1,rot_x_derivative1_derivative1,rot_x_derivative1_power2_derivative1,rot_y_derivative1,rot_y_derivative1_derivative1,rot_y_derivative1_power2_derivative1,rot_z_derivative1,rot_z_derivative1_derivative1,rot_z_derivative1_power2_derivative1


In [70]:
raw_func_img = nimg.load_img(func)
raw_func_img.shape

NameError: name 'func' is not defined

Group ICA

In [ ]:
canica = CanICA(n_components=20,
                memory="nilearn_cache", memory_level=2,
                verbose=10,
                mask_strategy='whole-brain-template',
                random_state=0)
canica.fit(func_filenames)

# Retrieve the independent components in brain space. Directly accessible through attribute `components_img_`.
canica_components_img = canica.components_img_

# components_img is a Nifti Image object, and can be saved to a file with the following line:

if not os.path.isdir('ICA'): 
    os.mkdir('ICA')
if not os.path.isdir('ICA/group'): 
    os.mkdir('ICA/group')
canica_components_img.to_filename('ICA/group/canica_resting_state_wo-sub.nii.gz')

In [ ]:
plot_prob_atlas(canica_components_img, title='All ICA components', output_file='ICA/group/canica_resting_state_wo-sub.png')

In [ ]:
n_columns = 2
n_ica = 20
nrows = int(n_ica / n_columns)
fig, axes = plt.subplots(nrows=nrows, ncols=n_columns, figsize=(20,60))

for i, cur_img in enumerate(iter_img(canica_components_img)):
    plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
                  cut_coords=3, 
                  colorbar=False, 
                  axes=axes[int(i / 2), int(i % 2)],
                  output_file='ICA/group/canica_resting_state_wo-sub_fig2.png')

This following loop does an ICA for each subject individualy

In [ ]:
for subject in layout.get_subjects():
    if subject not in ['07']:
        for session in layout.get_sessions(subject=subject):
            f = layout.get(task='rest', subject=['07'], session=session,
                           space='MNIPediatricAsym', desc='preproc', extension='nii.gz', 
                           return_type='filename')

            print(subject, session, f)

            canica = CanICA(n_components=20,
                            memory="nilearn_cache", memory_level=2,
                            verbose=1,
                            mask_strategy='whole-brain-template',
                            smoothing_fwhm=10,
                            random_state=0)
            canica.fit(f)


            if not os.path.isdir('ICA'):
                os.mkdir('ICA')
            if not os.path.isdir('ICA/individual'): 
                os.mkdir('ICA/individual')
            sub_folder = f'ICA/individual/{subject}'
            if not os.path.isdir(sub_folder): 
                os.mkdir(sub_folder)

            sub_path = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_ica.nii')

            # Retrieve the independent components in brain space. Directly ccessible through attribute `components_img_`.
            canica_components_img = canica.components_img_
            # components_img is a Nifti Image object, and can be saved to a file with he following line:
            canica_components_img.to_filename(sub_path)
            
            sub_path_fig1 = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_fig1_ica.png')

            plot_prob_atlas(canica_components_img, output_file=sub_path_fig1)
            
            n_columns = 2
            n_ica = 20
            nrows = int(n_ica / n_columns)
            fig, axes = plt.subplots(nrows=nrows, ncols=n_columns, figsize=(20,60))
            
            sub_path_fig2 = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_fig2_ica.png')

            for i, cur_img in enumerate(iter_img(canica_components_img)):
                plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
                              output_file=sub_path_fig2,
                              cut_coords=3, 
                              colorbar=False, 
                              axes=axes[int(i / 2), int(i % 2)])

In [73]:
func_filename='/home/carole/Bureau/nilearn/ICA/group/canica_resting_state_wo-sub.nii.gz'

In [80]:
from nilearn import image

data = image.load_img(func_filename).get_fdata()

In [81]:
data.mean()

0.00010725250814379367

In [82]:
data.max()

0.03219217973535315

In [83]:
data.min()

-0.019758241368737238